In [47]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer 
import json
import itertools
import datetime 
from sklearn.preprocessing import OneHotEncoder
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import json
import datetime
from collections import Counter

In [15]:
# path variables for KEVIN

path = "D:\Kevin Liang/Documents/1_UT_SENIOR/UT_AUSTIN_FALL_2017/EE_379K/Data_Science_Final_Project/Kickstarter_Data/"

In [16]:
def search_results(query):
    browser = webdriver.Firefox()
    browser.get(query)
    time.sleep(1)
    body = browser.find_element_by_tag_name('body')
    while True:
        elemsCount = browser.execute_script("return document.querySelectorAll('.stream-items > li.stream-item').length")

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            WebDriverWait(browser, 20).until(
                lambda x: x.find_element_by_xpath(
                    "//*[contains(@class,'stream-items')]/li[contains(@class,'stream-item')]["+str(elemsCount+1)+"]"))
        except:
            break

    errors = browser.find_elements_by_class_name('center')
    for error in errors:
        if (error.text == "Sorry! This person is no longer active on Kickstarter."):
            browser.quit()
            return 0, 0

    price = browser.find_elements_by_class_name('count')
    check = False
    for val in price:
        count = val.text
        count =count.encode('utf-8')
        if (check == True):
            num_backed= count
            check = False
        else:
            num_created = count
            check = True
            
    '''  
    projects = browser.find_elements_by_class_name('backed')
    for project in projects:
        text = project.text
        words = text.split()
        num_backed = str(words[1])
        #print num_backed
        browser.quit()
        
    '''  
    
    return int(num_backed), int(num_created)

In [17]:
def add_user_profile_data(df):
    user_num_backers = []
    user_num_creators = []
    
    for user_url in df['user_profiles']:
        b, c = search_results(user_url)
        user_num_backers.append(b)
        user_num_creators.append(c)
    
    df['user_num_backers'] = pd.Series(user_num_backers)
    df['user_num_creators'] = pd.Series(user_num_creators)
    
    return df

In [18]:
# data - Data set
# feature - feature that is going to be processed using nlp

def nlp_data(data, feature, total_words):
    all_words = []
    regex_word = "[^a-zA-Z]"
    lem = WordNetLemmatizer()

    for cell_data in data[feature]:
        words = re.sub(regex_word," ",cell_data).lower().split()
        words = [i for i in words if not i in stopwords.words('english')]
        # might change len of word
        words = [i for i in words if len(i) > 1]
        words = [lem.lemmatize(i, "v") for i in words]
        words = [lem.lemmatize(i) for i in words]
        all_words += words

    vectorizer = CountVectorizer(analyzer = "word", stop_words = "english", max_features = total_words)
    word_count = vectorizer.fit_transform(all_words).toarray()
    final_words = vectorizer.get_feature_names()
    
    return final_words

In [19]:
def preprocess_nlp(data, feature):
    word_list = []
    regex_word = "[^a-zA-Z]"
    lem = WordNetLemmatizer()
    for i in data[feature]:
        words = re.sub(regex_word," ",i).lower().split()
        words = [i for i in words if not i in stopwords.words('english')]
        # might change len of word
        words = [i for i in words if len(i) > 1]
        words = [lem.lemmatize(i, "v") for i in words]
        words = [lem.lemmatize(i) for i in words]
        word_list.append(words)
    data.drop(feature, axis = 1)
    data[feature] = word_list
    return data

In [57]:
def nlp(data, list_of_features, total_words):
    for i in list_of_features:
        final_words = nlp_data(data, i, total_words)
        data = preprocess_nlp(data,i)
        nlp_features = []
        for word in final_words:
            lst = []
            for x in data[i]:
                lst.append(x.count(word))
            nlp_features.append(lst)
            
        nlp_features = np.array(nlp_features)
        nlp_features_column = np.array(nlp_features).T
        nlp_feature_data = pd.DataFrame(data = nlp_features_column, columns = final_words)
        data = data.drop(i, axis = 1)
        data = pd.concat([data, nlp_feature_data], axis = 1)
    return data

In [21]:
# data - dataset as a DataFrame object
# list_of_features - features that need to be one hot encoded

def ohe_data(data, list_of_features):
    ohe = data[list_of_features]
    new_data = data.drop(list_of_features, axis = 1)
    ohe_features = pd.get_dummies(ohe)
    return pd.concat([new_data,ohe_features], axis = 1)

In [22]:
def json_creator_parser(data):
    new_feature = []
    for i in range(data["creator"].shape[0]):
        json_parsed_data = json.loads(data["creator"].iloc[i])
        new_feature.append(json_parsed_data['urls']['web']['user'])
        
    new_data = data.drop('creator', axis = 1)
    new_data["user_profiles"] = new_feature
    return new_data


In [23]:
def json_category_parser(data):
    new_feature_category = []
    new_feature_subcategory = []
    for i in range(data["category"].shape[0]):
        json_parsed_data = json.loads(data['category'].iloc[i])
        new_feature_category.append(json_parsed_data['slug'].split('/')[0])
        new_feature_subcategory.append(json_parsed_data['slug'].split('/')[1])
        
    new_data = data.drop('category', axis = 1)
    new_data["category"] = new_feature_category
    new_data["subcategory"] = new_feature_subcategory
    return new_data

In [55]:
data = pd.read_csv(path + "combined_data.csv", header = 0, index_col = 0)
data = data.drop(["photo", "urls","source_url","location"], axis = 1)
total_time = abs(data["launched_at"] - data["deadline"])
data["total_time"] = total_time
nlp_features = ["blurb","name"]
ohe_features = ["state","disable_communication","country","currency", "currency_trailing_code","is_starrable","current_currency","usd_type"]

In [56]:
data.shape

(157127, 20)

In [ ]:
data = nlp(data,nlp_features, 100)
data = ohe_data(data, ohe_features)
data = json_category_parser(data)
data = json_creator_parser(data)
#data = add_user_profile_data(data)
data


In [ ]:
#data.to_csv(path + "final_data.csv")

In [46]:
data.isnull().sum()

Unnamed: 0                   0
id                           0
name                         1
blurb                       10
goal                         0
pledged                      0
state                        0
disable_communication        0
country                      0
currency                     0
currency_trailing_code       0
deadline                     0
created_at                   0
launched_at                  0
is_starrable                 0
usd_pledged                  0
converted_pledged_amount     0
current_currency             0
usd_type                     0
creator                      0
category                     0
total_time                   0
dtype: int64

In [51]:
state = Counter(data["state"])
print state

Counter({'successful': 73688, 'failed': 70556, 'canceled': 8680, 'live': 3591, 'suspended': 612})


In [52]:
data.shape

(157127, 22)